This is a quick notebook to allow me to plot the locations of all the images used in my dataset in QGIS.

In [3]:
# Import data

import pathlib

import pandas as pd

features_path = pathlib.Path('/data/road_surface_classifier/features.csv')
df_feat = pd.read_csv(features_path).set_index('osm_id')
df_feat = df_feat.drop(columns=['wkt', 'x', 'y', 'ix', 'iy', 'length'])
df_feat

,highway,surface,lon,lat,object
osm_id,,,,,
391020748,primary,asphalt,-117.658360,33.596795,ca/2020/60cm/rgbir/33117/m_3311727_ne_11_060_2...
14290622,primary_link,asphalt,-115.147473,36.092396,nv/2019/60cm/rgbir/36115/m_3611563_ne_11_060_2...
240534097,motorway,asphalt,-78.739247,42.929866,ny/2019/60cm/rgbir/42078/m_4207803_sw_17_060_2...
684838122,secondary,asphalt,-112.012969,33.444280,az/2017/60cm/rgbir/33112/m_3311240_ne_12_h_201...
13567313,unclassified,asphalt,-88.843204,32.081660,ms/2020/60cm/rgbir/32088/m_3208858_nw_16_060_2...
...,...,...,...,...,...
13853468,unclassified,unpaved,-117.039719,47.376109,wa/2019/60cm/rgbir/47117/m_4711740_se_11_060_2...
14121032,unclassified,unpaved,-98.947994,42.185050,ne/2020/60cm/rgbir/42098/m_4209849_sw_14_060_2...
8757986,residential,unpaved,-74.787238,44.285550,ny/2019/60cm/rgbir/44074/m_4407442_se_18_060_2...


In [7]:
# Create dataset of points
from osgeo import ogr, osr

# Create SRS (EPSG:4326: WGS-84 decimal degrees)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

driver: ogr.Driver = ogr.GetDriverByName('GPKG')
ds: ogr.DataSource = driver.CreateDataSource('/data/road_surface_classifier/features_pts.gpkg')
layer: ogr.Layer = ds.CreateLayer('data', srs=srs, geom_type=ogr.wkbPoint)

osm_id_field = ogr.FieldDefn('osm_id', ogr.OFTInteger64)
highway_field = ogr.FieldDefn('highway', ogr.OFTString)
surface_field = ogr.FieldDefn('surface', ogr.OFTString)

layer.CreateField(osm_id_field)
layer.CreateField(highway_field)
layer.CreateField(surface_field)

feature_defn = layer.GetLayerDefn()

for _, row in df_feat.iterrows():
    feat = ogr.Feature(feature_defn)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint_2D(row['lon'], row['lat'])

    feat.SetGeometry(pt)
    feat.SetField('osm_id', row.name)
    feat.SetField('highway', row['highway'])
    feat.SetField('surface', row['surface'])
    layer.CreateFeature(feat)
    pt = None
    feat = None

layer = None # type: ignore
ds = None # type: ignore

In [6]:
# Create dataset of polygons from imagery
import json
from osgeo import gdal, ogr, osr

gdal.UseExceptions()
ogr.UseExceptions()

naip_img_path = pathlib.Path('/data/road_surface_classifier/imagery')
assert naip_img_path.is_dir()

# Create SRS (EPSG:4326: WGS-84 decimal degrees)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

driver: ogr.Driver = ogr.GetDriverByName('GPKG')
ds: ogr.DataSource = driver.CreateDataSource('/data/road_surface_classifier/features_polys.gpkg')
layer: ogr.Layer = ds.CreateLayer('data', srs=srs, geom_type=ogr.wkbPolygon)

osm_id_field = ogr.FieldDefn('osm_id', ogr.OFTInteger64)
highway_field = ogr.FieldDefn('highway', ogr.OFTString)
surface_field = ogr.FieldDefn('surface', ogr.OFTString)

layer.CreateField(osm_id_field)
layer.CreateField(highway_field)
layer.CreateField(surface_field)

feature_defn = layer.GetLayerDefn()

for osm_id, row in df_feat.iterrows():
    img_path = naip_img_path / str('%d.tif' % osm_id)
    assert img_path.exists()

    wgs84_extent = gdal.Info(str(img_path), format='json')['wgs84Extent']
    poly = ogr.CreateGeometryFromJson(json.dumps(wgs84_extent))

    feat = ogr.Feature(feature_defn)

    feat.SetGeometry(poly)
    feat.SetField('osm_id', row.name)
    feat.SetField('highway', row['highway'])
    feat.SetField('surface', row['surface'])
    layer.CreateFeature(feat)
    poly = None
    feat = None

layer = None # type: ignore
ds = None # type: ignore

Exception ignored in: <built-in function delete_DataSource>
Traceback (most recent call last):
  File "/tmp/ipykernel_57433/3662002279.py", line 16, in <cell line: 16>
RuntimeError: sqlite3_exec(CREATE TABLE gpkg_extensions (table_name TEXT,column_name TEXT,extension_name TEXT NOT NULL,definition TEXT NOT NULL,scope TEXT NOT NULL,CONSTRAINT ge_tce UNIQUE (table_name, column_name, extension_name))) failed: attempt to write a readonly database
